In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
! pip install transformers -q

     |████████████████████████████████| 501kB 6.7MB/s 
     |████████████████████████████████| 1.0MB 16.9MB/s 
     |████████████████████████████████| 3.7MB 32.9MB/s 
     |████████████████████████████████| 870kB 39.5MB/s 


In [0]:
import csv
import re
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
from fastai import *
from fastai.text import *
from fastai.callbacks import *
from fastai.metrics import *
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

from tqdm import tqdm
import torch
import torch.optim as optim

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

In [7]:
import os
os.chdir('/gdrive/My Drive/DEFINITION EXTRACTION/DEFT_Updated')
! ls

BERT_attention_kfold.ipynb	   rbert_ft
bert_cached_lm_510_deft.test.raw   roberta_cached_lm_510_deft.test.raw
bert_cached_lm_510_deft.train.raw  roberta_cached_lm_510_deft.train.raw
bert_ft				   RobertaLM_GB_kfold.ipynb
catboost_info			   roberta_tuned_test.npy
deft.test.raw			   roberta_tuned_train.npy
deft.train.raw			   runs
DualXLNet_attention_kfold.ipynb    sub
GCN_BERT_attention_kfold.ipynb	   task1_dev.csv
GCN_BERT_dependency_kfold.ipynb    task1_test.csv
GCN_BERT_final.ipynb		   task1_train.csv
gcn_dep.ipynb			   train_lm.py
gcn.ipynb			   vocab.txt
gcn_model.pkl			   WCL_cased_symbols.csv
glove.6B.200d.vec.txt		   XLNET_attention_kfold.ipynb
models				   XLNET_GCN_attention_kfold.ipynb


In [0]:
# model_name="bert-base-cased"
# model_name="roberta-base"
# model_name="xlnet-base-cased"
# model_name = 'distilbert-base-cased'

In [0]:
TEXT_COL = 'Sentence'
Y_COL = 'Label'
PTEXT_COL = 'SSentence'

In [0]:
import re
def multi_replace(chars_to_replace, char_to_place, from_string):
    for char in chars_to_replace:
        from_string = from_string.replace(char,char_to_place)
    return from_string

def preprocessing(sentence):
    remove_punct = multi_replace(',“”’.-{}[]()=+_!@#$%^&*<>?/|\\~`:;"','',sentence)
    return remove_punct

def load_data(file):
    df = pd.read_csv(file)
    # removing initial numbers
    df[TEXT_COL] = df[TEXT_COL].apply(lambda x : re.findall('^\s*\d*\s*\.?\s*(.*)',x)[0])
    # preprocessing
    # df[TEXT_COL] = df[TEXT_COL].apply(preprocessing)
    return df


In [11]:
train_df = load_data('task1_train.csv')
train_df.head(2)

,Subject,Sentence,Label
0,biology,Science includes such diverse fields as astron...,0
1,biology,"However , those fields of science related to t...",1


In [12]:
dev_df = load_data('task1_dev.csv')
dev_df.head(2)

,Subject,Sentence,Label
0,biology,It becomes clear from this definition that the...,0
1,biology,The scientific method is a method of research ...,1


In [13]:
test_df = load_data('task1_test.csv')
test_df.head(2)

,Subject,Sentence,Label\n
0,biology,Living things are highly organized and structu...,NaN
1,biology,The atom is the smallest and most fundamental ...,NaN


In [14]:
def subj(row):
  return '< ' + row['Subject'] + ' > ' + row[TEXT_COL] 

train_df[PTEXT_COL] = train_df.apply(subj, axis=1)
dev_df[PTEXT_COL] = dev_df.apply(subj, axis=1)
test_df[PTEXT_COL] = test_df.apply(subj, axis=1)

train_df.head()

,Subject,Sentence,Label,SSentence
0,biology,Science includes such diverse fields as astron...,0,< biology > Science includes such diverse fiel...
1,biology,"However , those fields of science related to t...",1,"< biology > However , those fields of science ..."
2,biology,"Thus , a museum of natural sciences might cont...",0,"< biology > Thus , a museum of natural science..."
3,biology,"In deductive reason , the pattern of thinking ...",0,"< biology > In deductive reason , the pattern ..."
4,biology,Deductive reasoning is a form of logical think...,1,< biology > Deductive reasoning is a form of l...


In [15]:
merged = pd.concat([train_df, dev_df])
merged.shape

(18315, 4)

## Transformer

In [0]:
 %tensorflow_version 1.x 
from transformers import *

## Preprocessing

In [17]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 64

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def tokenise_pad(df, col):
  X = [tokenizer.encode(x, add_special_tokens=True) for x in df[col]]
  X = pad_sequences(X, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  return X


X_train = tokenise_pad(train_df, PTEXT_COL)
X_dev = tokenise_pad(dev_df, PTEXT_COL)
X_test = tokenise_pad(test_df, PTEXT_COL)

Using TensorFlow backend.


In [0]:
def return_attention_masks(X):
  attention_masks = []
  for seq in X:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
  attention_masks = np.array(attention_masks)  
  return attention_masks

In [0]:
X_train_masks = return_attention_masks(X_train)
X_dev_masks = return_attention_masks(X_dev)
X_test_masks = return_attention_masks(X_test)

In [0]:
y_train = np.array(train_df[Y_COL])
y_dev = np.array(dev_df[Y_COL])

# Training

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [22]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
device

device(type='cuda', index=0)

In [0]:
from sklearn.utils.extmath import softmax
from sklearn.metrics import classification_report, f1_score
BATCH_SIZE = 32

def flat_accuracy(preds, labels):
    preds = softmax(preds)
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def flat_f1(preds, labels):
    preds = softmax(preds)
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat)


In [0]:
X_train = torch.tensor(X_train)
X_train_masks = torch.tensor(X_train_masks)
y_train = torch.tensor(y_train)

train_data = TensorDataset(X_train, X_train_masks, y_train)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

In [0]:
X_dev = torch.tensor(X_dev)
X_dev_masks = torch.tensor(X_dev_masks)
y_dev = torch.tensor(y_dev)

dev_data = TensorDataset(X_dev, X_dev_masks, y_dev)
dev_sampler = SequentialSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=BATCH_SIZE)

In [0]:
X_test = torch.tensor(X_test)
X_test_masks = torch.tensor(X_test_masks)

test_data = TensorDataset(X_test, X_test_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [31]:
from tqdm.notebook import tqdm

auto_model = None

del auto_model
auto_model = BertForSequenceClassification.from_pretrained('bert-base-cased', output_attentions=False, num_labels= len(np.unique(y_train)))
auto_model.to(device)

# Optimizer
num_total_steps = 1000
num_warmup_steps = 100
lr = 2e-5
param_optimizer = list(auto_model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
    'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
    'weight_decay_rate': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=lr)

best_loss = None

auto_model.train()  
tr_loss = 0
nb_tr_examples, nb_tr_steps = 0, 0
n_epochs = 4

for epoch in (range(n_epochs)):  
  # Training Loop
  auto_model.train()
  for step, batch in (enumerate(train_dataloader)):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    optimizer.zero_grad()
    outputs = auto_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

    loss, logits = outputs[:2]
    loss.backward()
    optimizer.step()
    
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1


  # Validation Loop
      
  auto_model.eval()
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  yt, yp = [], []
  for batch in dev_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
      outputs = auto_model(b_input_ids, 
                          token_type_ids=None,
                          attention_mask=b_input_mask,
                          labels= b_labels)        
      loss, logits = outputs[:2]
    logits = logits.detach().cpu().numpy()
    
    preds = softmax(logits)
    pred_ids = np.argmax(preds, axis=1).flatten()

    label_ids = b_labels.to('cpu').numpy()

    eval_loss += loss.item()
    eval_accuracy += flat_accuracy(logits, label_ids)

    yt = yt + label_ids.tolist()
    yp = yp + pred_ids.tolist()

    nb_eval_steps +=1
    
  print("Epoch {} | Train loss: {} | Validation Loss: {} ".format(epoch,
                                                                  tr_loss/nb_tr_steps,
                                                                  eval_loss/nb_eval_steps, 
                                                                ))
  if best_loss == None or best_loss > eval_loss/nb_eval_steps:
    best_loss = eval_loss/nb_eval_steps
    torch.save(auto_model.state_dict(), 'models/bert1_ft.pth')
    print(f'Saving model')
  

print(classification_report(yt, yp))   


Epoch 0 | Train loss: 0.3858908327439657 | Validation Loss: 0.3851492788504671 
Saving model
Epoch 1 | Train loss: 0.3321237861564308 | Validation Loss: 0.36160962752721926 
Saving model
Epoch 2 | Train loss: 0.28923487137793874 | Validation Loss: 0.41784365971883136 
Epoch 3 | Train loss: 0.25419108080251257 | Validation Loss: 0.5497275546745017 
              precision    recall  f1-score   support

           0       0.84      0.96      0.90       582
           1       0.89      0.62      0.73       272

    accuracy                           0.85       854
   macro avg       0.87      0.79      0.81       854
weighted avg       0.86      0.85      0.85       854



# Validation Analysis

In [32]:
auto_model = BertForSequenceClassification.from_pretrained('bert-base-cased', output_attentions=False, num_labels= len(np.unique(y_train)))
auto_model.load_state_dict(torch.load('models/bert1_ft.pth'))
auto_model.to(device)
auto_model.eval()
print('Loading best model')

yt, yp, p = [], [], []
for batch in dev_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch
  with torch.no_grad():
    outputs = auto_model(b_input_ids, 
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels= b_labels)        
    loss, logits = outputs[:2]
  logits = logits.detach().cpu().numpy()
  preds = softmax(logits)

  pred_ids = np.argmax(preds, axis=1).flatten()
  label_ids = b_labels.to('cpu').numpy()

  p = p + preds[:,1].tolist()
  yt = yt + label_ids.tolist()
  yp = yp + pred_ids.tolist()

Loading best model


In [33]:
dev_df['Probability'] = p
dev_df['Predicted'] = yp
dev_df['Loss'] = -dev_df['Label']*np.log2(dev_df['Probability']) - (1-dev_df['Label'])*np.log2(1-dev_df['Probability'])
dev_df = dev_df.sort_values(by='Loss', axis=0, ascending=False)
dev_df.head()

,Subject,Sentence,Label,SSentence,Probability,Predicted,Loss
93,biology,Some soils have no dominant particle size and ...,1,< biology > Some soils have no dominant partic...,0.002271,0,8.782654
163,biology,"Meiosis II , in which the second round of meio...",1,"< biology > Meiosis II , in which the second r...",0.002305,0,8.760772
650,government,The law requires driver ’s licenses and state ...,1,< government > The law requires driver ’s lice...,0.002532,0,8.625572
42,biology,A macrophage is a large phagocytic cell that e...,1,< biology > A macrophage is a large phagocytic...,0.003368,0,8.213726
378,psychology,"This step of rehearsal , the conscious repetit...",1,"< psychology > This step of rehearsal , the co...",0.003991,0,7.969112


In [34]:
c = 10
for i in dev_df['Sentence']:
  print(i)
  c-=1
  if c==0:
    break

Some soils have no dominant particle size and contain a mixture of sand , silt , and humus ; these soils are called loams .
Meiosis II , in which the second round of meiotic division takes place , includes prophase II , prometaphase II , and so on .
The law requires driver ’s licenses and state - issued identification cards ( DL / IDs ) to contain standardized anti - fraud security features , specific data , and machine - readable technology .
A macrophage is a large phagocytic cell that engulfs foreign particles and pathogens .
This step of rehearsal , the conscious repetition of information to be remembered , to move STM into long - term memory is called memory consolidation .
Bilateral symmetry involves the division of the animal through a sagittal plane , resulting in two mirror image , right and left halves , such as those of a butterfly ( [ link ] d ) , crab , or human body .
For example , the Clean Air Act sets air quality regulations but instructs states to design implementatio

In [35]:
from sklearn.metrics import confusion_matrix, f1_score
confusion_matrix(yt, yp), f1_score(yt, yp)

(array([[568,  14],
        [113, 159]]), 0.7146067415730338)

# Test Predictions

In [0]:
test_preds = []
auto_model.load_state_dict(torch.load('models/bert1_ft.pth'))
auto_model.eval()
print('Loading best model')

for batch in test_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask = batch
  with torch.no_grad():
    outputs = auto_model(b_input_ids, 
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        )        
    logits = outputs[0]
  logits = logits.detach().cpu().numpy()
  preds = softmax(logits)[:, 1]        
  test_preds = test_preds + preds.tolist()

Loading best model


In [0]:
test_df['Probablility'] = test_preds
test_df.to_csv('sub/submission_bert_final.csv', index=False)